# Data Wrangling and Visualization workbook
This workbook will be used to generate the data file that will be ready for use in the Event Based Model. Along the way, you will need to implement some of the data cleaning, data wrangling and data visualization skills that you learned in the demonstration notebook using the WHO cases data.  In the project notebook below, you will find instructions for what to do in each step followed by an empty code cell to enter your work. Don't forget that you will need to import the appropriate packages for this work.



In [1]:
# Put your import steps here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Step 1 - Load in ADNIMERGE
The first step we must do is to identify the measurements that will be used as features in the Event Based Model. From [Alex Young's original paper](https://academic.oup.com/brain/article/137/9/2564/2848155), the features that are to be included are: 
* Cerebrospinal Fluid (CSF) INNO-BIA AlzBio3 immunoassay ('INNO')
  * Amyloid Beta 1-42
  * phosphorylated tau
  * total tau
* Volumetric measurements from **1.5T** magnetic resonance imaging (MRI)
  * Whole brain volume
  * Ventricular volume
  * Entorhinal cortex volume
  * Hippocampal volume
  * Middle temporal cortex volume
  * Fusiform cortex volume 
  * Annualised whole brain atrophy between 0 and **12 months** using Boundary Shift Integral (BSI)
  * Annualised hippocampal atrophy between 0 and **12 months** using Boundary Shift Integral (BSI)
* Cognitive measures
  * Mini mental state examination (MMSE)
  * ADAS-COG13
  * Rey Auditory Verbal Learning Test (RAVLT)

Many of these features should be available in the ADNI MERGE dataset. The file is donwloaded from Teams and is called `adnimerge_ideas_merge_26may2022.csv`. We have included both the data dictionary and the methods so that you can better understand how this spreadsheet was created and what it represents. Please load in the ADNIMERGE spreadsheet. 
Here are some of the questions:
* Which column identifies the subject?
* Can you identify the features above in the column? 
* Are any of the features above missing?


In [5]:
# Your answer to Step 1
# Below put your code that will load up the ADNI MERGE spreadsheet
input_file = 'ADNI/adnimerge_ideas_merge_26may2022.csv'
df_raw = pd.read_csv(input_file)

/var/folders/qx/csfp5_y57613ykhb2sbzl1lh0000gn/T/ipykernel_68891/2173194490.py:4: DtypeWarning: Columns (20,105,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(input_file)


## Step 2 - Filter data
The event based model primarily works on cross-sectional data. In the data, the features you have identified from ADNIMERGE should be coming from the **baseline** visit and from scans acquired on a **1.5T** scanner.  Figure out how to filter the rows according to these two criteria. The resulting data frame should have 818 rows remaining.

In [85]:
# Your answer to Step 2
# Below put your code that will filter the ADNI MERGE spreadsheet
df_raw['FLDSTRENG'].value_counts()
df_raw['Month'].value_counts()
df_month = df_raw.loc[df_raw['Month'] == 0]
#print(df_month)
df_fld_month = df_month.loc[df_month['FLDSTRENG'] == '1.5 Tesla MRI']
print(df_fld_month.shape)

(818, 120)


## Step 3 - Merging data
You should have identified that some features from the original list that are missing in the ADNI MERGE spreadsheet. There is another spreadsheet in the data folder called `ucl_bsi_ideas_merge_26may2022.csv` where you will find the additional data.

In order to combine data you need to *merge* the two data sets. This involves finding key identifiers (the "on" columns) that will correspond to the same subject in both spreadsheets, so that the columns can be combined together in one data frame. Please remember that we only need BSI values from 0 to 12 months, so think about what filtering you will need to do with the new spreadsheet that you are loading in before performing the merge.

In [129]:
# Your answer to Step 3
# Below put your code that will merge the ADNI MERGE spreadsheet
# with other ADNI data availale.
input_file = 'ADNI/ucl_bsi_ideas_merge_26may2022.csv'
df_ucl = pd.read_csv(input_file)

#To filter out non-ADNI1 and MRI sequence != 1.5
df_ucl_one= df_ucl.groupby(['MRSEQUENCE'])
df_ucl_adni= df_ucl_one.get_group('ADNI1')

df_ucl_adni_mri= df_ucl_adni[df_ucl_adni['MRFIELD']==1.5]

#To filter only BSI 0-12m
filtering_values = ['m06','sc','bl']
df_ucl_filtered = df_ucl_adni_mri[df_ucl_adni_mri['VISCODE2'].isin(filtering_values)]

#print(df_ucl_filtered)

#Merge
df_merged = pd.merge(df_fld_month, df_ucl_filtered,
                       how='outer', on=['RID', 'VISCODE'],
                      )
df_merged = df_complete.reset_index(drop=True)
print(df_merged.shape)

(665, 98)


## Step 4 - Remove unneeded columns
After the merge, how many columns are there? There should be way more columns than what are needed for the event based model. So we can get rid of unwanted columns that we don't need for the replication. You should have:
* 14 feature columns from the list in Step 1
* Additional columns that should be include, such as:
    * subject identifiers (IDs, visit codes, exam dates, site, protocols)
    * demographics (Age, Gender)
    * diagnoses
    * _APOE_ genetic status
    * intracranial volume, to serve as a proxy for head size.
    * image quality control (QC) variables (such as `TEMPQC`,`REGRATING`)
    * anything else you find relevant. I chose to keep 49 columns, but you may have a few more or a few less. 

In [161]:
# Your answer to Step 4
# Below put your code that will remove 
# unnecessary columns from the data frame.

pd.set_option('display.max_columns', None)
#print(df_merged.columns.tolist())

df_complete_trial = df_merged.drop(columns = ['FSVERSION','EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'ADASQ4_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'mPACCdigit_bl', 'mPACCtrailsB_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'IMAGEUID_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'ABETA_bl', 'TAU_bl', 'PTAU_bl', 'Years_bl', 'Month_bl',
                                             'MOCA_bl', 'DT', 'VERSION', 'RUNDATE','STATUS','LONIUID','LONIUID_BASE',
                                             'VBSI','HBSI_R','HBSI_L', 'HBSI_T','HPACCEPT_R','HPACCEPT_L','ANN_HBSI']
                                   )
print(df_complete_trial.shape)

(665, 56)


In [163]:
df_complete_trial['TEMPQC']
df_complete_2 = df_complete_trial.query("TEMPQC == 'Pass'")

missing_values = df_complete_2.isnull()
missing_values.value_counts()
#missing_values

df_complete_3 = df_complete_2.dropna(axis=0, subset= ['TAU_INNO','ABETA_INNO','PTAU_INNO'])
df_complete_3.shape

(285, 56)

## Step 5 - Identify complete case data
The Event Based Model requires all features to be present for an observation to be included. Please remove any rows where one of the features that you plan to put in the EBM has missing data.

Please remove any data which failed QC check. The key column to check is `TEMPQC`.

Answer a few questions:
* For each variable, how many subjects had missing data? 
* How many subjects remain? 
* How do the subjects break down across diagnosis?
* Within diagnosis, how do they breakdown in terms of sex, age, APOE status?
* How do these numbers compare to the paper? 

This should be your final data set, which should consist of 283 participants.

In [158]:
# Your answer to Step 5
# Below put your code to remove missing data
# and answer the descriptive statsitics queries
# around the final data set

df_complete['TEMPQC']
df_complete = df_complete.query("TEMPQC == 'Pass'")

#missing_values = df_complete.isnull()
#missing_values.value_counts()
#missing_values

for column in missing_values:  
    print(column)
    print(missing_values[column].value_counts())
    print('----')

df_complete_2 = df_complete.dropna(axis=0, subset= ['TAU_INNO','ABETA_INNO','PTAU_INNO'])
df_complete_2.shape

#df_complete = df_complete.reset_index(drop=True)

#df_complete.info

RID
False    665
Name: RID, dtype: int64
----
COLPROT
False    665
Name: COLPROT, dtype: int64
----
ORIGPROT
False    665
Name: ORIGPROT, dtype: int64
----
PTID
False    665
Name: PTID, dtype: int64
----
SITE
False    665
Name: SITE, dtype: int64
----
VISCODE
False    665
Name: VISCODE, dtype: int64
----
EXAMDATE
False    665
Name: EXAMDATE, dtype: int64
----
DX_bl
False    665
Name: DX_bl, dtype: int64
----
AGE
False    665
Name: AGE, dtype: int64
----
APOE4
False    665
Name: APOE4, dtype: int64
----
FDG
True     333
False    332
Name: FDG, dtype: int64
----
ABETA_ELECSYS
True     350
False    315
Name: ABETA_ELECSYS, dtype: int64
----
TAU_ELECSYS
True     350
False    315
Name: TAU_ELECSYS, dtype: int64
----
PTAU_ELECSYS
True     350
False    315
Name: PTAU_ELECSYS, dtype: int64
----
CDRSB
False    665
Name: CDRSB, dtype: int64
----
ADAS11
False    665
Name: ADAS11, dtype: int64
----
ADAS13
False    660
True       5
Name: ADAS13, dtype: int64
----
ADASQ4
False    665
Name: ADASQ4, d

(285, 97)

## Step 6 - Data review
We have looked at the final data set at a group level, but let's actually visualise the data. For the various features that are to be included in the EBM, create some plots of your choice to look at how these values differ between the cognitively normal individuals and those showing evidence of cognitive impairment. You can break this latter group into mild cognitive impairment and Alzheimer's disease if you wish. You can also look at how this varies with _APOE_ status.

In [ ]:
# Your answer to Step 6
# Below put your code for visualising the data

